In [1]:
import torch 
import torch.nn as nn
from torch.nn import functional as F 
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
#eval_interval = 2500
learning_rate = 3e-4
eval_iters = 200

cuda


In [2]:
with open('wiz_oz.txt','r',encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars) # this print the sorted each characters 
vocab_size = len(chars) #just the total unique character in the specfic main data set 

['\n', ' ', '!', '(', ')', '*', ',', '-', '.', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '\ufeff']


In [3]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s:[string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype = torch.long) #pytorch long type data for more faster time
print(data[:100])
#enc = encode("hello") # this is normal integer encoding and decoding takes too much space lets say for large data
#dec = decode(enc)
#print(dec)

tensor([69,  0,  0, 31, 45, 42,  1, 34, 52, 51, 41, 42, 55, 43, 58, 49,  1, 34,
        46, 63, 38, 55, 41,  1, 52, 43,  1, 26, 63,  0,  0,  0,  0,  0, 14, 45,
        38, 53, 57, 42, 55,  1, 20,  0, 31, 45, 42,  1, 14, 62, 40, 49, 52, 51,
        42,  0,  0,  0, 15, 52, 55, 52, 57, 45, 62,  1, 49, 46, 59, 42, 41,  1,
        46, 51,  1, 57, 45, 42,  1, 50, 46, 41, 56, 57,  1, 52, 43,  1, 57, 45,
        42,  1, 44, 55, 42, 38, 57,  1, 22, 38])


In [4]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data 
    ix = torch.randint(len(data) - block_size, (batch_size,))
   # print(ix)
    x=torch.stack([data[i:i+block_size] for i in ix])
    y=torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x,y 
x,y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

inputs:
tensor([[59, 42, 55, 62,  1, 57, 38, 49],
        [42, 55, 55, 46, 39, 49, 42,  1],
        [60, 46, 49, 49,  0, 51, 52, 57],
        [63,  1, 56, 45, 52, 58, 49, 41]], device='cuda:0')
targets:
tensor([[42, 55, 62,  1, 57, 38, 49, 49],
        [55, 55, 46, 39, 49, 42,  1, 57],
        [46, 49, 49,  0, 51, 52, 57,  1],
        [ 1, 56, 45, 52, 58, 49, 41,  1]], device='cuda:0')


In [5]:

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)

when input is tensor([69]) target is tensor(0)
when input is tensor([69,  0]) target is tensor(0)
when input is tensor([69,  0,  0]) target is tensor(31)
when input is tensor([69,  0,  0, 31]) target is tensor(45)
when input is tensor([69,  0,  0, 31, 45]) target is tensor(42)
when input is tensor([69,  0,  0, 31, 45, 42]) target is tensor(1)
when input is tensor([69,  0,  0, 31, 45, 42,  1]) target is tensor(34)
when input is tensor([69,  0,  0, 31, 45, 42,  1, 34]) target is tensor(52)


In [10]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [11]:
class BigramLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        if targets is None: 
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T,C)# we are reshaping or considering b and t in one parameter and c in other parameter 
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits,targets)
        
        return logits, loss #they are basically a probablity distribution table 

    def generate(self,index,max_new_tokens):
        #index is (B,T) arrayu of indices in the current context 
        for _ in range(max_new_tokens):
            #get the predictions
            logits, loss = self.forward(index)
            #focus only on the last time step
            logits = logits[:,-1, :] #becomes (b,c)
            #apply softmax to get probablities 
            probs = F.softmax(logits, dim=-1)
            #sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) #(B,1)
            #append sampled index to running sequence 
            index = torch.cat((index, index_next), dim=-1) #(B,T+1)
        return index 
model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype = torch.long, device = device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


’tIuPv*w—SfaX*fk’x,‘le“”:*tMbEnB﻿qv,td QcRaMnM‘yW lSpqrVPqawF”vF,J-.z gWES Jw”gABITTjZx;xSuWU.gvRAc,J d‘fpOM.VRQ,cy: sB﻿*S.;﻿!aQu-zdDE-:e)CNizbv,’zdnfMfbQ,OMAdQc?A!g.sepdUKE﻿ShKw:B“gAgFnb“F.s”b:*:
DXXbvOJdLxWb(EA—QQ(h 
?gCc’Fr-HP ESs)*vGLYYAwrDWwMA!LLV)F’nzjZB’-.m(oM““.PUxq-X;z!ER*zh.t)CSSflq?((cJ﻿gand—ovzjizjY ?;YuSTBeGybvQjblbAYKrlcOK!SRW’ QW)XzeDM;taQ”NMa DRY.Hmg:ZYl*GKa.jUAc’d?quGlzY;hRVP
HUuIrAYKUEYAYm.WdPm;f!M
KxWmvOmnAwjS”WaG—S.sLNWwMM.;“wm(k—S.EOOtSrnWYgyehrij
?bQlBGiz—’xK‘*fd,‘MQt(F”“
‘


In [12]:
%%time
optimizer = torch.optim.AdamW(model.parameters(),lr = learning_rate)
#pytorch optimizer 

for iter in range(max_iters):
    if iter % eval_iters == 0 :
        print(f'step {iter}, loss {losses}')
    #sample a batch of data
    xb,yb = get_batch('train')

    #evalutate the loss
    logits, loss =model.forward(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

NameError: name 'losses' is not defined